# Job Post Scrutiniser

This script implements a solution to scrap the contents of a job post on a web page, analyse or examine the details then come up with the skills and qualifications required.

In [2]:
# imports

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


## The Web Scraper

In [28]:
from bs4 import BeautifulSoup
import requests


# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)


## Step 1: Create your prompts

In [ ]:

system_prompt = """
You are a smart assistant that analyzes the contents of a job post on a web page, and 
provides a summary of the skills and qualifications required, ignoring text that might be 
navigation related. Highlight the skills and qualifications in bullet points.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt = """
Here are the contents of a job post from a website.
Please analyze the contents and come up with a summary of the skills and qualifications 
required. Highlight the skills and qualifications separately in bullet points.


"""

## Step 2: Make the messages list

In [ ]:

def messages_for(webpage):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt + webpage}
    ]

## Step 3: Call OpenAI

In [ ]:

OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
openrouter = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=api_key)

def summarize(url, model):
    website = fetch_website_contents(url)
    response = openrouter.chat.completions.create(
        model = model,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [20]:
job_post_url = "https://job-boards.greenhouse.io/givedirectly/jobs/4558344005"
model = "claude-sonnet-4.6"

summarize(job_post_url, model)

"# Senior Software Engineer (Remote) at GiveDirectly\n\n## Role Summary\nGiveDirectly is seeking a Senior Software Engineer to join their remote team based in Nairobi, Kenya, supporting their mission of delivering cash transfers to people living in poverty worldwide.\n\n---\n\n## Hard Skills & Qualifications\n\n- Software engineering expertise at a **senior level**\n- Experience working in a **remote engineering environment**\n- Likely proficiency in **software development, coding, and system design** (implied by seniority)\n- Ability to work across **global, distributed teams** spanning multiple countries and time zones\n- Experience with **large-scale systems** (implied by organizational scale of 2M+ recipients across 15 countries)\n\n---\n\n## Soft Skills & Qualifications\n\n- **Analytical mindset** – aligned with GiveDirectly's candid and analytical culture\n- **Agility** – ability to work in a fast-paced, agile environment\n- **Candor** – comfort with direct, transparent communica

### A function to display this nicely in the output, using markdown

In [ ]:

def display_summary(url, model):
    summary = summarize(url, model)
    display(Markdown(summary))

## Step 4: Print the result

In [30]:
job_post_url = "https://www.wave.com/en/careers/job/5726127004/?source=LinkedIn"
display_summary(job_post_url, model)

# Engineering Manager - Database Reliability at Wave

## Summary
Wave is seeking an experienced Engineering Manager to lead their Database Reliability team, ensuring high availability and scalability of database infrastructure across their fast-growing financial services platform operating across Africa.

---

## Required Qualifications

- Minimum **5 years of experience** as a manager supporting database or storage teams
- Must be located in a **timezone between GMT-1 and GMT+3**
- Must be based in one of the talent hub countries: **UK, Spain, Kenya, or Ghana**, or in an operating market in Africa
- Willingness to **travel to Africa once per year** (~6 days)

---

## People & Management Skills

- Demonstrable **people management experience**: coaching, performance evaluation, mentorship, and career development
- **Project management experience**: prioritization, tracking schedules, deadlines, and cross-team communication
- **Process design experience**: on-call rotation management, incident response, postmortems, load testing, and capacity planning
- **Hiring experience**: predicting staffing needs, designing interview loops, evaluating and closing candidates

---

## Technical Skills

- Proven track record of running **mission-critical databases in production** (replication, backups, migrations, HA/DR)
- Proficiency with **Kubernetes** and container orchestration (DB workloads in K8s is a bonus)
- Experience with **cloud infrastructure**, preferably **GCP**, or Data Center environments
- Familiarity with **Postgres** and/or **CockroachDB**
- Knowledge of **monitoring, cost optimization, and performance tuning**

---

## Preferred/Bonus Skills

- Experience with backend development in **Python 3**
- Familiarity with **Terraform** for infrastructure management
- Experience with **GraphQL** API layer
- Knowledge of **TypeScript/React** (web frontend)
- Experience scaling databases to handle high-volume workloads